#Deployment 

## install library

In [1]:
!pip install fastapi
!pip install pydentic
!pip install pyngrok
!pip install unicorn
!pip install fastapi nest-asyncio pyngrok uvicorn
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## import library

In [2]:
from flask import Flask
from pyngrok import ngrok
import nest_asyncio
import uvicorn
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from inspect import Parameter
from itertools import count
from typing import Any, Dict
from urllib.request import Request
from fastapi import Body,FastAPI,Request
from matplotlib.pyplot import text
from pydantic import BaseModel, Field as PydanticField
from pydantic.fields import Field
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch



In [3]:
%cd /content/drive/MyDrive/Data_set/recommender_system

/content/drive/MyDrive/Data_set/recommender_system


In [4]:
! ngrok authtoken "2CgE5efiOHm9YtS0PAOAIiyZKmO_6s95xgufUcucd1Mu2ZTPx"

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [20]:
class model:
    def __init__(self):
        self.tv = TfidfVectorizer()
        self.loaded_model = LogisticRegression()
        self.loaded_model = pickle.load(open('finalized_model.sav', 'rb'))
        self.tv = pickle.load(open('tfidf.pickle', 'rb'))
        X = pickle.load(open('train_comment_features.pickle', 'rb'))
        self.tv.fit(X)
        self.df=pd.read_csv('book.csv')
        self.book_matrix = self.df.pivot_table(index='userID', columns='title', values='rating')


    def getCo(self,bookName):
        # Collect the rate Similarities:
        book_rate = self.book_matrix[bookName]
        similar_to_air_book = self.book_matrix.corrwith(book_rate)
        return similar_to_air_book

    def recommend(self,bookName,feedBack):
      if feedBack =="Positive":
        return self.getCo(bookName).idxmax()
      else :
        return self.getCo(bookName).idxmin()




    def sa(self,text):
        print(text)
        t=self.tv.transform([text])
        return self.loaded_model.predict(t)




In [25]:

app =FastAPI()

ml = model()
df=pd.read_csv('book.csv')

tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines


books=[]
recommed_book=[]



@app.post('/')
async def home(info: Request):
    req_info = await info.json()
    print()
    
    intent_NAme = req_info["queryResult"]["intent"]["displayName"]
    text = ""
    if(intent_NAme=="TEST"):
       text ="this is test intent"
    elif (intent_NAme=="MovieName"):
       text ="Move name is avatar"


    return {"fulfillmentMessages": [
      {
        "text": {
          "text": [
            text
          ]
        }
      }
    ]}

@app.post('/SA')
async def SA(info: Request):
    req_info = await info.json()
    print()
    intent_NAme = req_info["queryResult"]["queryText"]
    text = str(ml.sa(intent_NAme))
    books.append([recommend,text])
    return {"fulfillmentMessages": [
      {
        "text": {
          "text": [
            text
          ]
        }
      }
    ]}
@app.post('/Diolgpt')
async def ranam(info: Request):

    with open('HISTORY.txt','r') as f:
        History_id=f.readlines()
        History_id=int(History_id)
        print(History_id)
    req_info = await info.json()
    print()
    intent_NAme = req_info["queryResult"]["queryText"]
    text = df.sample()['title'].values[0]

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(intent_NAme + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([History_id, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)
    text=tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

    with open('HISTORY.txt','w') as f:
      History_id+=1
      f.write(History_id)
    return {"fulfillmentMessages": [
      {
        "text": {
          "text": [
            text
          ]
        }
      }
    ]}
@app.post('/recommend')
async def recommend(info: Request):

    req_info = await info.json()
    intent_NAme = req_info["queryResult"]["intent"]["displayName"]
    intent_NAme2 = req_info["queryResult"]["queryText"]
    text = str(ml.sa(intent_NAme))
    if(intent_NAme=="recommender followup"):
      with open('temp.txt','r') as f:
        recommed_book=f.readlines()
        print(recommed_book)
      text_sa = str(ml.sa(intent_NAme2))[1:-1]
      print(text_sa)
      text =ml.recommend(recommed_book[0],text_sa)
      recommed_book=text
    elif (intent_NAme=="recommender"):
      text = df.sample()['title'].values[0]
      recommed_book =text
    with open('temp.txt','w') as f:
      f.write(recommed_book)
    
    
    return {"fulfillmentMessages": [
      {
        "text": {
          "text": [
            text
          ]
        }
      }
    ]}


In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: http://5323-35-204-49-246.ngrok.io


INFO:     Started server process [1464]
INFO:uvicorn.error:Started server process [1464]
INFO:     Waiting for application startup.
INFO:uvicorn.error:Waiting for application startup.
INFO:     Application startup complete.
INFO:uvicorn.error:Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
INFO:uvicorn.error:Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))